# Section 5. Spatial Data - Color plots, 3D plots

#### Instructor: Pierre Biscaye

The content of this notebook draws on material from UC Berkeley's Spatial Data Analysis [course](https://docs.google.com/document/d/1oC10pjyeBQTenQazCpaB8Lx1b5PC1SR3WFiPgCtXqcs/edit?tab=t.0) notes by [Jaecheol Lee](https://sites.google.com/view/jaecheollee).

### Learning Objectives 
    
* Learn different ways of plotting spatial fields
* Plotting spatial data with color and 3-D plots

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
%matplotlib inline

In many cases you will want to plot information over space, rather that just spatial objects. There are many ways to approach this.

Here we will cover two ways: 3-D plots and colormaps.

In [ ]:
# Import some libraries
# for 3d plotting
from mpl_toolkits.mplot3d import Axes3D
# color
from matplotlib.colors import LinearSegmentedColormap

Like we did for the heatmap before, let's first define the space we want to examine by describing the axes. We define the axes by specifying their end points and the number of points in between. We describe the axes in each dimension as follows.

Begin by defining the width of a grid, $\delta$ (for simplicity, we'll just assume resolution is same in the x and y directions). 

Then using this parameter, we describe the axis in each dimension using a vector for each.

In [ ]:
delta = 1
x_axis = np.arange(start=-5, stop=5 + delta, step=delta)
y_axis = np.arange(start=-5, stop=5 + delta, step=delta)

These two vectors describe a space where there are grid points centered at 121 locations (draw a picture for yourself if you don’t know why). 
It’s easy to define functions over this space using map algebra. 
To do this, we construct two matrices, one that describes the location of each grid point in the x-dimension, 
and one that describes the locations of each grid point in the y-dimension. 
The command meshgrid takes as inputs the two vectors that define the space and outputs these two matrices.

In [ ]:
X_grid, Y_grid = np.meshgrid(x_axis, y_axis)
X_grid

In [ ]:
Y_grid

To avoid confusion, we usually refer to elements in a matrix using the indices (i, j) and we refer to the locations in 2D space using the coordinates (x, y) or (lon, lat).

The two matrices `X_grid` and `Y_grid` are useful because we can do map algebra on them to generate new functions (fields) that are defined at each location in x and y. For example, start with a simple plane:

In [ ]:
Q_field = X_grid + 2 * Y_grid

### Plotting a field in a 3D plot.

Let's visualize this field over space using a 3D plot.

In [ ]:
fig = plt.figure()
# tell matplotlib our axes are three-dimensional
ax = fig.add_subplot(1, 1, 1, projection='3d')
# here we plot the surface
ax.plot_surface(X_grid, Y_grid, Q_field)
plt.show()

The height on the z-axis already tells us the values of the Q field. We can add color to make it easier to visualize.

We can also spin the 3-D figure around by running the code below. Note that elev angles up/down, and azim rotates clockwise/counterclockwise.
``` 
ax.view_init(elev=30, azim=-125)
```

In [ ]:
fig = plt.figure()
# tell matplotlib our axes are three-dimensional
# this is similar to a question in Lab 2
ax = fig.add_subplot(1, 1, 1, projection='3d')
# here we plot the surface
ax.plot_surface(X_grid, Y_grid, Q_field, cmap='viridis')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=30, azim=-125)
plt.show()

Let's define a new field and explore other types of 3-D figures.

$$Z = 40 - 0.4X^2 + 2Y - 0.5Y^2$$

There are very many possibilities!!

In [ ]:
Z_field = 40 -0.4 * X_grid**2 + 2 * Y_grid - 0.5 * Y_grid**2

fig = plt.figure(figsize=(10, 3))
ax0 = fig.add_subplot(1, 3, 1, projection='3d')
ax0.plot_wireframe(X_grid, Y_grid, Z_field, color='grey')
ax1 = fig.add_subplot(1, 3, 2, projection='3d')
ax1.plot_surface(X_grid, Y_grid, Z_field, cmap='bone')
ax2 = fig.add_subplot(1, 3, 3, projection='3d')
ax2.plot_trisurf(X_grid.flatten(),
                 Y_grid.flatten(),
                 Z_field.flatten(),
                 cmap='bone')
plt.show()

### Plotting a field in 2D using imshow

Most maps we produce are 2D, so it is useful to think of how to display data this way. `matplotlib` includes a function `imshow` which colors spatial data according to specified values.

In [ ]:
fig, ax = plt.subplots(ncols=2)
im = ax[0].imshow(Q_field, cmap='coolwarm')
# ax tells colorbar() which subplot to plot on
fig.colorbar(im, ax=ax[0])
im = ax[1].imshow(Z_field, cmap='coolwarm')
fig.colorbar(im, ax=ax[1])
plt.show()

Examine your y-axis in the plot. Is it labeled correctly? What's wrong with it? Try the commands

```python 
ax.set_xticklabels(x_axis)
ax.set_yticklabels(y_axis)
```

What does passing the argument `origin='lower'` do to `ax.imshow()`?


In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(Q_field, cmap='coolwarm', origin='lower')
ax.set_xlabel('X')
ax.set_ylabel('Y')
# show all ticks
ax.set_xticks(np.arange(len(x_axis)))
ax.set_yticks(np.arange(len(y_axis)))
# set all tick labels
ax.set_xticklabels(x_axis)
ax.set_yticklabels(y_axis)
fig.colorbar(im, ax=ax)
plt.show()

There are many colormaps you can use, and you can also define your own.

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
nodes = [0, 0.5, 1]  # positions for each color from 0-1: 0 to vmin, 1 to vmax
color_scheme = ['white', 'yellow', 'red']  # corresponds to nodes
custom_cmap = LinearSegmentedColormap.from_list(
    'custom_name', list(zip(nodes, color_scheme)))
custom_cmap.set_under('gray')  # set values under vmin to gray
custom_cmap.set_over('red')  # set values over vmax to red

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(7, 3))
# 0th subplot
im = axes[0].imshow(Q_field, cmap=custom_cmap, origin='lower')
plt.colorbar(im, ax=axes[0])
# 1st subplot
im = axes[1].imshow(Q_field, cmap='jet', origin='lower')
plt.colorbar(im, ax=axes[1])
# label all axes, title subplots, etc.
for ax in axes:
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    # this allows for rendering of latex math formula
    ax.set_title('$Q = X + 2Y$')
    # show all ticks
    ax.set_xticks(np.arange(len(x_axis)))
    ax.set_yticks(np.arange(len(y_axis)))
    # set all tick labels
    ax.set_xticklabels(x_axis)
    ax.set_yticklabels(y_axis)
plt.tight_layout()
plt.show()

### Plotting in 2D using contour plots

Let's display the Z field using a contour plot. There are two main contour-plotting commands, so we will do this on a two subplot figure.

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2)
ax0.contour(X_grid, Y_grid, Z_field, cmap='coolwarm')
ax1.contourf(X_grid, Y_grid, Z_field, cmap='coolwarm')
for ax in (ax0, ax1):
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    # this allows for rendering of latex math formula
    ax.set_title('$Z = 40 - 0.4X^2 + 2Y - 0.5Y^2$')
plt.tight_layout()
plt.show()